# 1. 事前にハイパーパラメータのチューニングを行います

## パラメータチューニング手法
- grid search
- random
- bayse optimzation
※ チューニングには時間がかかります

In [8]:
# チューニング関数への入力
class TrainingConfig:
    csv_path = './sample_data/train_T1.csv' # csvのパス
    task = 'regression' # regression or clastaring
    tuning_method = 'random-search' # チューニングメソッド
    tadk_id = 'T1_pred'

In [2]:
from train import train
conf = TrainingConfig
train(conf)

ModuleNotFoundError: No module named 'polars'

In [ ]:
# 2. 予測と評価

In [ ]:
from xgb import xgb_pred_val